In [28]:
from utils import dump_data, render_data, fx
from creds import qualtrics_token
import pandas as pd
import psycopg2
from datetime import datetime
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline  
import time
import sys
from creds import rds_url, rds_port, rds_db, rds_username, rds_password, user, password
import pandas as pd
import pandas.io.sql as sqlio

In [2]:
dump_data("SV_9ucQAGOaxR6Dl7U")

{"result":{"progressId":"ES_1FgYs4ndLAt2PnE","percentComplete":0.0,"status":"inProgress"},"meta":{"requestId":"79754636-ccef-46e5-b0a1-16bdd936ef29","httpStatus":"200 - OK"}}
{"result":{"fileId":"b68ed69f-9d78-4edf-9774-8c7452ce2362-def","percentComplete":100.0,"status":"complete"},"meta":{"requestId":"a5d6d83e-1848-4e1b-89a6-9b4c5e3a3588","httpStatus":"200 - OK"}}
True


In [6]:
d = pd.read_csv("/Users/zive/GDrive/research/yourfeed_analysis/data/SV_9ucQAGOaxR6Dl7U.csv/yourfeed_pilot_v2.csv")
d = d.iloc[2:]

In [113]:
#d[['Duration (in seconds)','yourfeed','rid','crt_race']]

In [8]:
conn = psycopg2.connect(
    host=rds_url,
    port=rds_port,
    dbname=rds_db,
    user=rds_username,
    password=rds_password)

In [76]:
stim_dat = sqlio.read_sql_query("select * from stimulus where experiment_id='5123' and stim != 'introCard[dwell]'", conn)

In [13]:
demo_dat = sqlio.read_sql_query("select * from demographics where experiment_id='5123'", conn)

In [77]:
stim_dat['dwell'] = stim_dat['stim'].str.contains("dwell")
stim_dat['stim'] = stim_dat['stim'].apply(lambda x:re.findall(r'\d+', x)[0])
#stim['response_y'] = stim['response_y'].fillna(-1).astype(int)+1

In [109]:
stim=pd.merge(stim_dat[stim_dat['dwell']],stim_dat[['user_id','stim','response']][~stim_dat['dwell']], on=['user_id','stim'], how='left')
stim['response_y'] = stim['response_y'].fillna(-1).astype(int)+1
stim['response_x'] = stim['response_x'].astype(int)

In [112]:
print(np.mean(stim[stim['response_y']==0]['response_x']))
print(np.mean(stim[stim['response_y']==1]['response_x']))

1407.4666666666667
2457.6923076923076
